In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import  glob
from scipy.spatial import ConvexHull
from Constants import Const
import joblib
from pointcloud_utils import *
from scipy.spatial.distance import cdist, pdist, squareform
import matplotlib as mpl
import torchio as tio

import plotly.graph_objects as go
from pointcloud_utils import *
%matplotlib notebook

# from ReaderWriter import BetterDicomReader, dicom_reader_from_ids
#from multiprocessing import cpu_count
# import typing
import os
# from queue import *
# from tqdm import tqdm
# from threading import Thread
import pickle
import simplejson
# import open3d as o3d
import pydicom
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

In [4]:
# import pydicom
# import os
# files = glob.glob('../data/DICOMs/R01/**/**/**')
# dicoms = []
# for f in files:
#     p = pydicom.dcmread(f)
#     if p and not f.endswith('.dcm'):
#         newfile = f + '.dcm'
#         test = os.rename(f,newfile)
#         print(test,f,newfile)
#         print('_______')
# glob.glob('../data/DICOMs/R01/**/**/**')

In [5]:
def get_finished_pids(root=None):
    if root is None:
        root = Const.pointcloud_dir
    files = glob.glob(root + 'pclouds_*.json')
    pids = []
    for file in files:
        pid = file.replace( root+'pclouds_','').replace('.json','')
        if pid.isnumeric():
            pids.append(int(pid))
        else:
            print('bad pid',pid)
    return pids

len(get_finished_pids())

0

In [6]:
def get_all_dicom_ids(root = None):
    if root is None:
        root = Const.unprocessed_dicoms
    files = glob.glob(root+'*/')
    ids = []
    for f  in files:
        pid = f.replace(root,'').replace('/','')
        if pid.isnumeric():
            ids.append(int(pid))
        else:
            print('bad pid',pid)
    return ids
get_all_dicom_ids()

[1054079696,
 1072572079,
 1079757401,
 1087308891,
 1099927508,
 1103015749,
 1107681010,
 1108642427,
 1138096463,
 1143391354,
 1164546699,
 1167168001,
 1173728658,
 1178044145,
 1194116893,
 1235569621,
 1265845118,
 1269213210,
 1276736352,
 2774318802,
 2776638846,
 2803120404,
 2804890849,
 2815583275,
 2817169157,
 2843895295,
 2865890865,
 2867468906,
 2875588687,
 2883823678,
 2889102751,
 2894996073,
 2908060983,
 2929571068,
 2932807221,
 2939740989,
 2983776095,
 2989874876,
 3028333367,
 3035721150,
 3045110595,
 3045556976,
 3045918834,
 3049758970,
 3071491330,
 3077525807,
 3081060090,
 3099145083,
 3100266114,
 3108161676,
 3125978990,
 3151449158,
 3192749693,
 3192926363,
 3194171650,
 3196236080,
 3205005928,
 3225956079,
 3235750820,
 3239454379,
 3256350141,
 3283538700,
 3316448321,
 3321571409,
 3327830751,
 3346286228,
 3355839884,
 3366888610,
 3370207907,
 3383739521,
 3442548822,
 3514335131,
 3563849182,
 3694073920,
 3816960111,
 3832820671,
 4017119917,

In [7]:
def load_patient_folder(root,pid,as_dict=True):
    files = glob.glob(root+str(pid) + '/**/**')
    entry = {}
    pids = set([])
    pid2 = False
    for f in files:
        p = pydicom.dcmread(f)
        pid2 = str(p.PatientName)
        pids.add(pid2)
        dtype = str(p.SOPClassUID.name)
        subentry = entry.get(dtype,[])
        subentry.append(p)
        entry[dtype] = subentry
    if len(pids) > 1:
        print('mutilple ids',pids)
    if as_dict:
        return {int(pid2): entry}
    return entry, pid2

def sample_pids(sample_size=10,skip_finished=True):
    potential_ids = get_all_dicom_ids()
    if skip_finished:
        to_skip = get_finished_pids()
        potential_ids = [pid for pid in potential_ids if pid not in to_skip]
    if sample_size is not None and sample_size < len(potential_ids):
        potential_ids = potential_ids[:sample_size]
    return potential_ids

def load_dicoms(skip_finished=True,sample_size=10):
    root = Const.unprocessed_dicoms
    to_skip = []
    potential_ids = sample_pids(sample_size=sample_size,skip_finished=skip_finished)
    dicom_files = {}
    files = []
    for pid in potential_ids:
        pentry,new_pid = load_patient_folder(root,pid,as_dict=False)
        if str(new_pid) != str(pid):
            print('id mismatch',pid,new_pid)
        dicom_files[int(new_pid)] = pentry

    return dicom_files

def load_dicoms_by_ids(pids):
    dicom_files = {}
    for pid in pids:
        pentry, newpid = load_patient_folder(Const.unprocessed_dicoms,pid,as_dict=False)
        if str(newpid) != str(pid):
            print('id mismatch',pid,newpid)
        dicom_files[int(newpid)] = pentry
    return dicom_files

# load_dicoms_by_ids([4646292867, 9643148771])
# dicom_files = load_dicoms(sample_size=30)
# dicom_files.keys()

In [8]:
def get_element(ds,string,default=False):
    try:
        return ds.data_element(string).value
    except:
        return default
    
def fix_roi_name(roi):
    roi = roi.lower()
    if roi not in Const.organ_associations:
        if 'gtv' in roi or 'rtv' in roi:
            if 'gtvn' in roi or 'node' in roi or 'nodal' in roi:
                return 'gtvn'
            else:
                return 'gtv'
        if 'ptv' in roi:
            return 'ptv'
        if 'ctv' in roi:
            return 'ctv'
        return Const.organ_associations.get(roi.replace(' ','_').replace('cavity_oral','oral_cavity'),roi)
    return Const.organ_associations.get(roi,roi)

def read_rt_struct(rtstruct,contour_dict = None,associations=None):
    #this should read an rtstruct file, clean the names
    #returns a dict of {roi: [pointcloud,pointclouds...]}
    #multiple pointclouds if there are different contours that are name varaints of a single organ (list gtv)
    
    #pass contour dict if there are mutliple rt struct files?
    if contour_dict is None:
        contour_dict = {}
    if associations is None:
        associations=Const.organ_associations
        

    rseq_list = get_element(rtstruct,'ROIContourSequence',[])
    roi_list = get_element(rtstruct,'StructureSetROISequence',[])
    if len(rseq_list) < 1 or len(roi_list) < 1:
        return False
    assert(len(rseq_list) == len(roi_list))
    for rcseq,roi in zip(rseq_list,roi_list):
        try:
            
            name = fix_roi_name(roi.ROIName)
            number = roi.ROINumber
            if 'ContourSequence' in rcseq:
                cs = rcseq.ContourSequence
                #each contourSequence is at a different z-height, so Imma just merge them
                contours = [np.array(s.ContourData).reshape(-1,3) for s in rcseq.ContourSequence if len(s.ContourData) > 0]
                contours = np.vstack(contours)
                curr_entry = contour_dict.get(name,[])
                curr_entry.append(contours)
                contour_dict[name] = curr_entry
        except Exception as e:
            print('error in read_rt_struct',e)
    return contour_dict

# fix_roi_name('dsfsdgtv')
# dicom_files[3194171650]['RT Dose Storage'][0]
# read_rt_struct(rt_struct_file)

In [9]:

def get_dose_pointcloud(dose,threshold=0):
    data = dose['data']
    if threshold is None:
        threshold = data.min()
    contour = np.argwhere(data.T > threshold)
    (cx,cy,cz) = np.where(data.T > threshold)
    point_values = data.T[cx,cy,cz].ravel()
    return {'points': contour, 'values': point_values}

def points_to_spatial(points, location, gridsize):
    scale = np.array(gridsize)
    corner = np.array(location)
    newpoints = points*scale + corner
    return  newpoints

def dose_pcloud(dose,scale=None,**kwargs):
    cloud = get_dose_pointcloud(dose,**kwargs)
    spacing = dose['spacing']
    location = dose['location']
    thickness = dose['thickness']
    if thickness is None or thickness <= 0: #missing, assume 3
        thickness = 3
    grid =np.array([spacing[0],spacing[1],thickness])
    if scale is not None:
        grid = grid*scale
    coords = points_to_spatial(cloud['points'],location,grid)
    return {'coordinates': coords, 'values': cloud['values']}

def pointcloud_subset(pcloud, contour,values,slice_thickness=2):
    #given a pointcloud pc and a pointlcoud countour(that is a contour) calculate points in pc that are inside the contour
    zvals =  np.unique(contour[:,2])
    zmax = zvals.max()
    zmin = zvals.min()
    below = pcloud[:,2] < zmax
    above = pcloud[:,2] > zmin
    within = below & above
    pc = pcloud[within]
    vals = values[within]
    good_points = []
    good_values = []
    for z in zvals:
        cslice = contour[contour[:,2] == z]
        if cslice.shape[0] < 4:
            continue
        poly = Polygon(cslice)
        in_subset = (np.absolute(pc[:,2] - z) <= slice_thickness/2)
        point_subset = pc[in_subset]
        val_subset = vals[in_subset]
        is_good = [poly.contains(Point(p)) for p in point_subset]
        good_points.extend(point_subset[is_good])
        good_values.extend(val_subset[is_good])
    if len(good_points) < 1:
        return [[],[]]
    return [np.stack(good_points),np.array(good_values)]

def pointcloud_overlap(pcloud, contour,slice_thickness=0):
    #given a pointcloud pc and a pointlcoud countour(that is a contour) calculate points in pc that are inside the contour
    zvals =  np.unique(contour[:,2])
    zmax = zvals.max()
    zmin = zvals.min()
    below = pcloud[:,2] < zmax
    above = pcloud[:,2] > zmin
    within = below & above
    pc = pcloud[within]
    max_overlap = 0
    for z in zvals:
        cslice = contour[contour[:,2] == z]
        if cslice.shape[0] < 4:
            continue
        poly = Polygon(cslice)
        in_subset = (np.absolute(pc[:,2] - z) <= slice_thickness/2)
        point_subset = pc[in_subset]
        for p in point_subset:
            point = Point(p)
            if poly.contains(point):
                dist = poly.exterior.distance(point)
                if dist > max_overlap:
                    max_overlap = dist
    return max_overlap

def get_dose_roi_intersections(pdict,roilist=None):
    contours = pdict['contours']
    dosecloud = pdict['dose_pointcloud']['coordinates']
    dosevals = pdict['dose_pointcloud']['values']
    thickness = pdict['dose']['thickness']
    if roilist is None:
        roilist = list(contours.keys())
    results = {}
    for roi in roilist:
        contourlist = contours.get(roi,[])
        rlist = []
        dlist = []
        for contour in contourlist:
            points,vals = pointcloud_subset(dosecloud,contour,dosevals,slice_thickness=thickness)
            if len(points) > 0:
                rlist.append(points)
                dlist.extend(list(vals))
        if len(rlist) > 0:
            results[roi] = {'coordinates': np.vstack(rlist), 'dose_values': np.array(dlist)}
    return results

def pointcloud_distance_batched(pc1,pc2,batch_size=200,metric='euclidean',check_overlap=True):
    #get two pointcloud arrays, checks distance between each pair of points
    #returns the smallest distance (inter-organ distance)
    if np.array(pc1).shape[0] < 2 or np.array(pc2).shape[0] < 2:
        return False
    
    if check_overlap:
        overlap = pointcloud_overlap(pc1,pc2)
        if overlap > 0:
            return -overlap
        
    pc1_batches = np.array_split(pc1,batch_size,axis=0)
    min_dist = 10000000
    for batch in pc1_batches:
        if len(batch)  > 1:
            batch_dists = cdist(batch,pc2,metric)
            batch_min = batch_dists.min()
            if batch_min < min_dist:
                min_dist = batch_min
    if min_dist == 10000000:
        return 0
    return min_dist

def pcloud_dist_worker(args):
    if len(args) != 2:
        return 0
    (pc1, pc2) = args
    if pc1 is None or pc2 is None or len(pc1) < 1 or len(pc2) < 1:
        return 0
    return pointcloud_distance_batched(np.vstack(pc1),np.vstack(pc2))

def interorgan_distances(pclouds,sources=None,targets=None,n_jobs=-2):
    #assumes a dict of {roi: Nx3 array of poitns, roi2...}
    keys = list(pclouds.keys())
    if sources is None:
        sources = keys
    if targets is None:
        targets = keys

    distances = []
    for source in sources:
        scloud = pclouds.get(source)
        if scloud is None:
            dists = [0 for i in targets]
        else:
            dists = joblib.Parallel(n_jobs=n_jobs)( joblib.delayed(pcloud_dist_worker)((scloud,pclouds.get(target,[]))) for target in targets)
        distances.append(dists)
    return  distances


# dose_pcloud(pdict_spatial[0]['dose'])

In [ ]:
class DicomProcessor():

    
    def __init__(self,dicom_files = None, path = None, max_count = 40):
        if dicom_files is None:
            dicom_files = self.read_dicoms(path)
        self.dicoms=dicom_files    
        self.uids = list(dicom_files.keys())
        self.active_uid =  self.uids[0]
        
    def read_dicoms(self,path,max_count=None):
        files = glob.glob(path)
        dicom_files = {}
        for f in files:
            p = pydicom.dcmread(f)
            pid = str(p.PatientName)
            entry = dicom_files.get(pid,{})
            dtype = str(p.SOPClassUID.name)
            if dtype not in ['RT Dose Storage','RT Structure Set Storage']:
                continue
            subentry = entry.get(dtype,[])
            subentry.append(p)
            entry[dtype] = subentry
            dicom_files[pid] = entry
            if max_count is not None and len(dicom_files) >= max_count:
                break
        return dicom_files
    
    def get_patient(self,uid=None):
        if uid is None:
            uid = self.active_uid
        return self.dicoms.get(uid)
    
    def get_all_patients(self):
        plist = [(uid,self.dicoms.get(uid)) for uid in self.uids]
        return plist
    
def get_dicom_spatial(ds):
    position = get_element(ds,'ImagePositionPatient')
    orientation = get_element(ds,'ImageOrientationPatient')
    slice_thickness = get_element(ds,'SliceThickness')
    loc = get_element(ds,'SliceLocation')
    spacing = get_element(ds,'PixelSpacing')
    data = ds.pixel_array
    entry = {
        'data': data,
        'position': position,
        'orientation': orientation,
        'location': loc,
        'spacing': spacing,
        'slice_thickness': float(slice_thickness) if slice_thickness else 3.0,
    }
    return entry

def get_dicom_identifiers(ds):
    #these should alway be present
    study_uid = ds.StudyInstanceUID
    series_uid = ds.SeriesInstanceUID
    class_uid = str(ds.SOPClassUID.name) #shouldn't be optional
    pid = str(ds.PatientID)
    entry = {
        'patient_id': pid,
        'study_uid': study_uid,
        'series_uid': series_uid,
        'type': class_uid,
    }
    return entry

def check_consistency(ct_images,identifiers):
    identifiers = get_dicom_identifiers(ct_images[0]) if identifiers is None else identifiers
    #just check for no
    flag = True
    for ct in ct_images:
        temp_identifiers = get_dicom_identifiers(ct)
        for k,v in temp_identifiers.items():
            if 'series' not in k and 'type' not in k and identifiers[k] != v:
                print('consistency error',k,v,identifiers[k])
                flag = False
    return flag

def save_individual_patient(pdict,folder=None):
    folder = Const.pointcloud_dir if folder is None else folder
    fname = folder + 'pclouds_' + str(pdict['patient_id'])
    contour_name = folder + 'contours_' + str(pdict['patient_id'])
    to_keep = ['patient_id',
               'study_uid',
               'series_uid',
               'spacing','contours',
               'contour_pointclouds',
               'missing_rois',
               'extra_rois'
              ]
    entry = {k:pdict.get(k) for k in to_keep}
    with open(fname+'.json','w') as f:
        simplejson.dump(entry,f,default=np_converter)
    with open(contour_name+'.json','w') as f2:
        simplejson.dump(pdict['dose_pointcloud'],f2,default=np_converter)
    
    return True



def process_patient(pdict,save=True):
    dose_data = pdict.get('RT Dose Storage')
    rt_structs = pdict.get('RT Structure Set Storage')
    entry = get_dicom_identifiers(dose_data[0])
    is_good = check_consistency(dose_data,entry)
    is_good = is_good and check_consistency(rt_structs,entry)
    if not is_good:
        return False
    contours ={}
    for rtstruct in rt_structs:
        contours = read_rt_struct(rtstruct,contour_dict=contours)

    entry['contours'] = contours
    all_rois = set(contours.keys())
    entry['missing_rois'] = [c for c in Const.organ_list + ['gtv','gtvn','ptv','ctv'] if c not in all_rois]
    entry['extra_rois'] = [c for c in all_rois if c not in Const.organ_list + ['gtv','gtvn','ctv','ptv']]
    print('missing rois',entry['missing_rois'])
    print('extra rois:',entry['extra_rois'])
    
    dose_spatial = []
    for dsi in dose_data:
        spatial = get_dicom_spatial(dsi)
        dose_spatial.append(spatial)
            
    topi = lambda array,i: np.min([d['position'][i] for d in array])
    dose_corner = (topi(dose_spatial,0),topi(dose_spatial,1),topi(dose_spatial,2))
    
    dose_data = dose_spatial[0]['data']
    #idk if this works yet, but they mentioned summing up voxels for muliple dose files
    #i dont see that in the current data so idk
    if len(dose_spatial) > 1:
        for d in dose_spatial[1:]:
            dose_data = dose_data + d['data']
            
    dose_entry = {
        'location': dose_corner,
        'spacing': dose_spatial[0]['spacing'],
        'orientation': dose_spatial[0]['orientation'],
        'thickness': dose_spatial[0]['slice_thickness'],
        'data': dose_data,
        'data_shape': dose_data.shape
    }
    
    dose_cloud = dose_pcloud(dose_entry) #coordinates, values
    entry['dose'] = dose_entry
    entry['dose_pointcloud'] = dose_cloud

    print('running getting contours',entry['patient_id'])
    valid_rois = [c for c in all_rois if c in Const.organ_list[:] + ['gtv','gtvn','ptv','ctv']]
    entry['contour_pointclouds'] = get_dose_roi_intersections(entry,roilist=valid_rois)
    print('got contours, missing',[c for c in valid_rois if c not in entry['contour_pointclouds'].keys()])
    print(dose_entry['thickness'],dose_entry['spacing'])
    if save:
        save_individual_patient(entry)
    return entry 


def process_dicoms_spatial(dicom_files=None,path=None,skip_finished=True,calc_dists=True):
    if path is None:
        path = Const.unprocessed_dicoms + '**/**/**'
    dp = DicomProcessor(dicom_files=dicom_files,path=path)
    plist = []
    dfilename = Const.small_dist_json
    try:
        with open(dfilename,'r') as f:
            dfile = simplejson.load(f)
            distances = dfile['distances']
    except:
        distances = {}
        dfile = {'distances': distances} 
        
    sources = ['gtv','gtvn']
    targets = Const.organ_list[:]
    dfile['rowOrder'] = sources
    dfile['colOrder'] = targets
    for (pid,pfiles) in dp.get_all_patients():
        if int(pid) in [3192926363]:
            continue
        print('__________\n')
        print('new patient',pid)
        p = process_patient(pfiles)
        if p:
            plist.append(p)
            if calc_dists:
                pdists = interorgan_distances(p['contours'],sources=sources, targets = targets)
                distances[pid] = pdists.astype('float16')
                print('distances',distances)
                try:
                    with open(dfilename,'w') as f:
                        dfile['distances'] = distances
                        simplejson.dump(dfile,f,default=np_converter)
                except Exception as e:
                    print('issue saving at pid',pid)
                    print(e)
        else:
            print('issue getting ',pid)
    return plist

#this code runs the original processing

test = ['placeholder','placehodler']
while len(test) > 1:
    test = process_dicoms_spatial(dicom_files=load_dicoms(sample_size=10))
test
# process_dicoms_spatial(dicom_files=load_dicoms_by_ids([4646292867, 9643148771]))

In [11]:
Const.organ_list

['hyoid',
 'mandible',
 'brainstem',
 'oral_cavity',
 'glottis',
 'thyroid',
 'cricoid',
 'cricopharyngeal_muscle',
 'esophagus',
 'glnd_submand_l',
 'glnd_submand_r',
 'genioglossus_m',
 'glottis',
 'hard_palate',
 'soft_palate',
 'ipc',
 'spc',
 'mpc',
 'parotid_l',
 'parotid_r',
 'larynx',
 'supraglottic_larynx',
 'lips_lower',
 'lips_upper',
 'ant_digastric_l',
 'ant_digastric_r',
 'mastoid_l',
 'mastoid_r',
 'medial_pterygoid_l',
 'medial_pterygoid_r',
 'lateral_pterygoid_l',
 'lateral_pterygoid_r',
 'buccinator_l',
 'buccinator_r',
 'masseter_l',
 'masseter_r',
 'post_digastric_l',
 'post_digastric_r',
 'sternocleidomastoid_l',
 'sternocleidomastoid_r',
 'mylogeniohyoid',
 'post_scalene_l',
 'post_scalene_r',
 'ant_scalene_l',
 'ant_scalene_r',
 'spinal_cord',
 'tongue',
 'pituitary']

In [ ]:
#check to see that they are all encoded properly
def load_pcloud(pid):
    fname =Const.pointcloud_dir + 'pclouds_' + str(int(pid)) + '.json'
    try:
        with open(fname,'r') as f:
            data = simplejson.load(f)
        return data
    except Exception as e:
        print(e)
        return False
    
    
def validate_data(return_data=False,fix=True):
    pids = get_finished_pids()
    good = []
    bad = []
    for pid in pids:
        pdata = load_pcloud(pid)
        if pdata:
            if return_data:
                good.append(pdata)
            else:
                good.append(pid)
        else:
            bad.append(pid)
    print('bad pids',bad)
    if fix:
        process_dicoms_spatial(dicom_files=load_dicoms_by_ids(bad))
    return good, bad
# validate_data()

In [ ]:
def show_extra_rois(pids=None):
    if pids is None:
        pids = get_finished_pids()
    def big_if_true(name,array):
        array = list(set(array))
        if len(array) > 1:
            print(name,' | '.join(array))
            print()
    all_candidates = set([])
    all_extra = set([])
    for pid in pids:
        pdata = load_pcloud(pid)
        if pdata:
            print('_____',pid,'________')
            all_rois = pdata['contours'].keys()
            needed_rois = pdata['contour_pointclouds'].keys()
            missing = [c for c in Const.organ_list[:] + ['gtv','gtvn','ptv','ctv'] if c not in all_rois]
            extra = [c for c in all_rois if c not in Const.organ_list[:] + ['gtv','gtvn','ptv','ctv'] ]
            candidates = []
            for r in extra:
                for c in Const.organ_list + ['submand','smg','spinal','gland']:
                    if c.replace('_l','').replace('_r','') in r:
                        candidates.append(r)
                        all_candidates.add(r)
            for e in extra:
                all_extra.add(e)
            big_if_true('needed',needed_rois)
            big_if_true('candidates',candidates)
            big_if_true('extra',extra)
            print('__________')
    print('all candidates', ' | '.join(all_candidates))
    print('____')
    print('all extra', ' | '.join(all_extra))
# show_extra_rois()

In [ ]:
#note: ask about stuff like 'dose 62--[cgy] fs 66.8gy, etc and if that is likea ptv or something
def get_extra_rois_candidates(pids=None):
    if pids is None:
        pids = get_finished_pids()
  
    candidates = {}
    for pid in pids:
        pdata = load_pcloud(pid)
        if pdata:
            print('_____',pid,'________')
            all_rois = pdata['contours'].keys()
            extra = [c for c in all_rois if c not in Const.organ_list[:] + ['gtv','gtvn','ptv','ctv'] ]
            for r in extra:
                for c in Const.organ_list + ['submand','smg','spinal','gland']:
                    if c.replace('_l','').replace('_r','') in r:
                        candidates[r] = c
    for k,v in Const.organ_associations.items():
        candidates[k] = v
    return candidates
# get_extra_rois_candidates()

In [ ]:
def full_distances(dfilename=None,skip_finished=True,root=None):
    if dfilename is None:
        dfilename = Const.full_dist_json
    if root is None:
        root = Const.pointcloud_dir
    try:
        with open(dfilename,'r') as f:
            dfile = simplejson.load(f)
            distances = dfile['distances']
    except:
        distances = {}
        dfile = {'distances': distances}
    processed_patients = glob.glob(root+ 'pclouds_*.json')
    sources = ['gtv','gtvn'] + Const.organ_list[:]
    targets = Const.organ_list[:]
    dfile['rowOrder'] = sources
    dfile['targetOrder'] = targets
    for file in processed_patients:
        pid = int(file.replace( root+'pclouds_','').replace('.json',''))
        if skip_finished and pid in distances:
            continue
        print('running',pid)
        try:
            with open(file,'r') as f:
                pdict = simplejson.load(f)
            contours = {k: [np.array(vv) for vv in v] for k,v in pdict['contours'].items()}
            print('contours loaded')
            pdists = interorgan_distances(contours,sources=sources,targets=targets)
            distances[pid] = pdists 
            with open(dfilename,'w') as f:
                dfile['distances'] = distances
                simplejson.dump(dfile,f,default=np_converter)
            print(pid,'done',len(distances))
            
        except Exception as e:
            print('error reading distances',e)
            print('______________')
    return distances
    
#json with gtv/gtvn + organ -> organ distances
#row order is gtv,gtvn,Const.organ_list, col: const.organ_list
full_distances()

In [ ]:
def small_distances(dfilename=None,skip_finished=True,root=None):
    if dfilename is None:
        dfilename = Const.small_dist_json
    if root is None:
        root = Const.pointcloud_dir
    try:
        with open(dfilename,'r') as f:
            dfile = simplejson.load(f)
            distances = dfile['distances']
    except:
        distances = {}
        dfile = {'distances': distances}
    processed_patients = glob.glob(root+ 'pclouds_*.json')
    sources = ['gtv','gtvn']
    targets = Const.organ_list[:]
    dfile['rowOrder'] = sources
    dfile['targetOrder'] = targets
    for file in processed_patients:
        pid = int(file.replace( root+'pclouds_','').replace('.json',''))
        if skip_finished and pid in distances:
            continue
        print('running',pid)
        try:
            with open(file,'r') as f:
                pdict = simplejson.load(f)
            contours = {k: [np.array(vv) for vv in v] for k,v in pdict['contours'].items()}
            print('contours loaded')
            pdists = interorgan_distances(contours,sources=sources,targets=targets)
            distances[pid] = pdists 
            with open(dfilename,'w') as f:
                dfile['distances'] = distances
                simplejson.dump(dfile,f,default=np_converter)
            print(pid,'done',len(distances))
            
        except Exception as e:
            print('error reading distances',e)
            print('______________')
    return distances
    
#json with gtv/gtvn + organ -> organ distances
#row order is gtv,gtvn,Const.organ_list, col: const.organ_list
small_distances()

In [ ]:
def full_distances(dfilename=None,skip_finished=True,root=None):
    if dfilename is None:
        dfilename = Const.full_dist_json
    if root is None:
        root = Const.pointcloud_dir
    try:
        with open(dfilename,'r') as f:
            dfile = simplejson.load(f)
            distances = dfile['distances']
    except:
        distances = {}
        dfile = {'distances': distances}
    processed_patients = glob.glob(root+ 'pclouds_*.json')
    sources = ['gtv','gtvn'] + Const.organ_list[:]
    targets = Const.organ_list[:]
    dfile['rowOrder'] = sources
    dfile['targetOrder'] = targets
    for file in processed_patients:
        pid = int(file.replace( root+'pclouds_','').replace('.json',''))
        if skip_finished and pid in distances:
            continue
        print('running',pid)
        try:
            with open(file,'r') as f:
                pdict = simplejson.load(f)
            contours = {k: [np.array(vv) for vv in v] for k,v in pdict['contours'].items()}
            print('contours loaded')
            pdists = interorgan_distances(contours,sources=sources,targets=targets)
            distances[pid] = pdists 
            with open(dfilename,'w') as f:
                dfile['distances'] = distances
                simplejson.dump(dfile,f,default=np_converter)
            print(pid,'done',len(distances))
            
        except Exception as e:
            print('error reading distances',e)
            print('______________')
    return distances
    

def get_dose_roi_intersections(pdict,roilist=None):
    contours = pdict['contours']
    dosecloud = np.array(pdict['dose_pointcloud']['coordinates'])
    dosevals = np.array(pdict['dose_pointcloud']['values'])
    thickness = pdict['dose']['thickness']
    if roilist is None:
        roilist = list(contours.keys())
    results = {}
    for roi in roilist:
        contourlist = contours.get(roi,[])
        rlist = []
        dlist = []
        for contour in contourlist:
            contour = np.array(contour)
            points,vals = pointcloud_subset(dosecloud,contour,dosevals,slice_thickness=thickness)
            if len(points) > 0:
                rlist.append(points)
                dlist.extend(list(vals))
        if len(rlist) > 0:
            results[roi] = {'coordinates': np.vstack(rlist), 'dose_values': np.array(dlist)}
    return results

def fix_patients(root=None):
    if root is None:
        root = Const.pointcloud_dir
    processed_patients = glob.glob(root+ 'pclouds_*.json')
    fixed = []
    skipped=[]
    for file in processed_patients:
        pid = int(file.replace( root+'pclouds_','').replace('.json',''))
        print('running',pid)
        try:
            with open(file,'r') as f:
                entry = simplejson.load(f)
            if 'contour_pointclouds' not in entry:
                print('fixing...')
                entry['contours'] = {k: [np.array(vv) for vv in v] for k,v in entry['contours'].items()}
                all_rois = set(entry['contours'].keys())
                valid_rois = [c for c in all_rois if c in Const.organ_list[:] + ['gtv','gtvn']]
                entry['dose'] = {'thickness': 3}
                print('starting contouring')
                entry['contour_pointclouds'] = get_dose_roi_intersections(entry,roilist=valid_rois)
                print('finished contouring')
                save_individual_patient(entry)
                print('saved successful')
                print('__________')
                fixed.append(pid)
            else:
                skipped.append(pid)
        except Exception as e:
            print(e)
            print('_________________')
            skipped.append(pid)
    return fixed,skipped

fix_patients()

In [ ]:
# import json
# with open('../data/DICOMS/ProcessedPatients/pclouds_1334966362.json','r') as f:
#     test = json.load(f)
# test

In [ ]:
test.keys()